## import libraries

In [75]:
from pytube import YouTube


download_dir = "youtube-download"
slide_dir = "youtube-slide"

## Take YouTube Video link as input and Download the Video

In [76]:
link = input("Enter the YouTube Video Link Containing Slide: ")
link = link.split("&")[0]
yt = YouTube(link)
try:
    title = yt.title.split()[0] + ".mp4"
except:
    title = "current_video.mp4"

try:
    try:
        yt.streams.filter(adaptive=True, only_video=True).order_by('resolution').desc().first().download(download_dir, title)
    except KeyError as e:
        yt.streams.filter(progressive=True).order_by('resolution').desc().first().download(download_dir, title)
except AttributeError as ae:
    print("Attribute Error try another video")

'Continuous.mp4'